# Lib install

In [1]:
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q langchain
!pip install -q sentence_transformers
!pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00


In [1]:
# On Collab, kernel restart may be needed after installing accelerate for it to register properly

In [2]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.30.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0


# Lib imports

In [3]:
import torch
from torch import cuda, bfloat16
import transformers
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader

#Parameters

In [4]:
model_id = 'NousResearch/Llama-2-7b-chat-hf'
hf_auth = 'paste_your_token_here' # Hug Face token - repace paste_your_token_here with your token

# Model & Pipeline

In [5]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

Model loaded on cuda:0


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

In [8]:
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [9]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [10]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,
    max_new_tokens=512,
    repetition_penalty=1.1,
    do_sample=True
)

# Initial result w/o Web sources

In [11]:
res = generate_text("Who was Sargeras?")
print(res[0]["generated_text"])

Who was Sargeras?
 Unterscheidung zwischen Sargeras und dem Titelfiguren des Films "Sargerass" auf YouTube.


In [12]:
#The model seems to have no sensible context about Sargeras.

In [13]:
llm = HuggingFacePipeline(pipeline=generate_text)

# Ensuring pipeline returns the same result as above
llm(prompt="Who was Sargeras?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Who was Sargeras?\n Unterscheidung zwischen Sargeras und dem Titelfiguren des Films "Sargerass" auf YouTube.'

# Adding Web sources

In [14]:
#Passing a couple of links to WoW Wiki to scrape information about Sargeras
web_links = ["https://wowpedia.fandom.com/wiki/Sargeras","https://wowpedia.fandom.com/wiki/Titan","https://wowpedia.fandom.com/wiki/Burning_Legion","https://wowpedia.fandom.com/wiki/Pantheon"]

loader = WebBaseLoader(web_links)
documents = loader.load()

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [16]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

vectorstore = FAISS.from_documents(all_splits, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

# Results with Web info

In [25]:
chat_history = []

query = "Who was Sargeras?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Sargeras created a planet, Mardum, and bound the souls of all the demons he fought to it.
Telepathy (able to commune with his servants, such as Mannoroth and Archimonde, from the Nether)
Illusion (can show visions and speak in a compelling angelic voice)[42]
Invulnerability against mortal weapons, and limited immunity to immortal weapons (Broxigar's Axe of Cenarius was able to scratch him, but not to any great effect, while the powers of the Demon Soul had no effect).
Ability to enter mortal bodies (remained within Aegwynn's womb for nearly a millennium) and take control of them (Medivh).
The strength of 100 dragons is merely a fraction of Sargeras' strength;[43] his strength nearly matches the power of a portal closing, and was able to cut a planet in two with a single blow of his sword.[9]
In comparison to Sargeras, Mannor

In [28]:
#Retrieving the helpful answer piece:
print(result['answer'][result['answer'].rfind("Helpful Answer:"):])

Helpful Answer: Sargeras was a Titan, a powerful being created by the Great Dark Beyond to bring order to the universe. He was known as the Defender and was considered the noblest of all the Titans. He was originally altruistic and just, but he eventually discovered that some demons were using the dark energies of the Void to threaten the Pantheon's work. Despite this, he continued his work, suspecting the existence of even more powerful malevolent beings hiding behind the veil of the Void.


In [27]:
#The answer is sensible and based on Wiki information.

# Conversation

In [30]:
chat_history = [(query, result["answer"])]

query = "What did he look like?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Sargeras created a planet, Mardum, and bound the souls of all the demons he fought to it.
Telepathy (able to commune with his servants, such as Mannoroth and Archimonde, from the Nether)
Illusion (can show visions and speak in a compelling angelic voice)[42]
Invulnerability against mortal weapons, and limited immunity to immortal weapons (Broxigar's Axe of Cenarius was able to scratch him, but not to any great effect, while the powers of the Demon Soul had no effect).
Ability to enter mortal bodies (remained within Aegwynn's womb for nearly a millennium) and take control of them (Medivh).
The strength of 100 dragons is merely a fraction of Sargeras' strength;[43] his strength nearly matches the power of a portal closing, and was able to cut a planet in two with a single blow of his sword.[9]
In comparison to Sargeras, Mannor

In [32]:
#Retrieving the helpful answer piece:
print(result['answer'][result['answer'].rfind("Helpful Answer:"):])

Helpful Answer: According to the lore, Sargeras was a massive being with molten bronze skin and eyes that glowed like stars. He had wings spread wide enough to block out the sun and a sword that could cut through a planet.


In [33]:
#Answer seems to properly provide response based on previous context and the follow-up question.

In [34]:
#Checking sources of answers:
print(result['source_documents'])

[Document(page_content="Sargeras created a planet, Mardum, and bound the souls of all the demons he fought to it.\nTelepathy (able to commune with his servants, such as Mannoroth and Archimonde, from the Nether)\nIllusion (can show visions and speak in a compelling angelic voice)[42]\nInvulnerability against mortal weapons, and limited immunity to immortal weapons (Broxigar's Axe of Cenarius was able to scratch him, but not to any great effect, while the powers of the Demon Soul had no effect).\nAbility to enter mortal bodies (remained within Aegwynn's womb for nearly a millennium) and take control of them (Medivh).\nThe strength of 100 dragons is merely a fraction of Sargeras' strength;[43] his strength nearly matches the power of a portal closing, and was able to cut a planet in two with a single blow of his sword.[9]\nIn comparison to Sargeras, Mannoroth and Archimonde were as fleas.[43] This could be a reference to either power or height.", metadata={'source': 'https://wowpedia.fan